In [5]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OllamaEmbeddings(model="mistral"),
)
retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
  ("system", "Answer the question based only on the following context: {context}"),
  ("human", "Question: {question}")
])

model = ChatOllama(model="mistral", temperature=0)

retrieval = RunnableParallel({"context": retriever, "question": RunnablePassthrough()})

chain = retrieval | prompt | model | StrOutputParser()

chain.invoke("where did harrison work?")

' Harrison worked at Kensho, but this information is unrelated to the fact that bears like to eat honey.'

In [6]:
from operator import itemgetter

prompt = ChatPromptTemplate.from_messages([
  ("system", "Answer the question based only on the following context: {context}"),
  ("human", "Question: {question}"),
  ("system", "Answer in the following language: {language}")
])

retrieval = RunnableParallel({
  "context": itemgetter("question") | retriever, 
  "question": itemgetter("question"),
  "language": itemgetter("language")
})

chain = retrieval | prompt | model | StrOutputParser()

chain.invoke({"question": "where did harrison work", "language": "italian"})

' Risposta: Harrison ha lavorato a Kensho.\n\nEnglish translation: Harrison worked at Kensho.'

# `Conversational Retrieval Chain`

In [7]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel
from langchain.prompts.prompt import PromptTemplate

def _combine_documents(docs):
    """Combine a list of documents into a single string."""
    doc_strings = [format_document(doc, DEFAULT_DOCUMENT_PROMPT) for doc in docs]
    return "\n\n".join(doc_strings)

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(
    """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
    Chat History:{chat_history}
    Follow Up Input: {question}
    Standalone question:
    """
)

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template("{page_content}")

ANSWER_PROMPT = ChatPromptTemplate.from_messages([
    ("system", "Answer the question based only on the following context: {context}"),
    ("human", "Question: {question}"),
])

_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(chat_history=lambda x: get_buffer_string(x["chat_history"]))
    | CONDENSE_QUESTION_PROMPT
    | model
    | StrOutputParser(),
)

_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}

conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | model

In [8]:
conversational_qa_chain.invoke({"question": "where did harrison work?", "chat_history": []})

AIMessage(content=" Harrison works at Kensho. However, the information given in the question does not provide any details about Harrison's job role or connection to bears or honey.")

In [9]:
conversational_qa_chain.invoke(
    {
        "question": "where did he work?",
        "chat_history": [
            HumanMessage(content="Who wrote this notebook?"),
            AIMessage(content="Harrison"),
        ],
    }
)

AIMessage(content=" Harrison worked at Kensho. The context does not provide any information about Harrison's job duties or tasks related to bears or honey.")

## `With Memory and returning source documents`

In [10]:
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnableLambda

memory = ConversationBufferMemory(return_messages=True, output_key="answer", input_key="question")

loaded_memory = RunnablePassthrough.assign(chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"))

standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | model
    | StrOutputParser(),
}

retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}

final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}

answer = {
    "answer": final_inputs | ANSWER_PROMPT | model,
    "docs": itemgetter("docs"),
}

chain = loaded_memory | standalone_question | retrieved_documents | answer

In [12]:
inputs = {"question": "where did harrison work?"}
result = chain.invoke(inputs)
result

{'answer': AIMessage(content=" Harrison works at Kensho. However, the information given in the question does not provide any details about Harrison's job role or connection to bears or honey."),
 'docs': [Document(page_content='bears like to eat honey'),
  Document(page_content='harrison worked at kensho')]}

In [13]:
memory.save_context(inputs, {"answer": result["answer"].content})
memory.load_memory_variables({})

result = chain.invoke({"question": "but where did he really work?"})
result

{'answer': AIMessage(content=" The context does not provide enough information to determine the exact location or company where Harrison works. Harrison's occupation was mentioned in relation to a statement about bears and honey, but there is no connection between Harrison's work and bears or honey in the given context."),
 'docs': [Document(page_content='bears like to eat honey'),
  Document(page_content='harrison worked at kensho')]}